In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # used for splitting training and testing data

torch.manual_seed(1)

In [2]:
plt.figure(figsize=(8,5))

# time steps per batch of data
seq_length = 100

time_steps = np.linspace(0, np.pi, seq_length + 1)

data = pd.read_csv('../Datasets/SectionData.csv')

x = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

<Figure size 576x360 with 0 Axes>

In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, ouput_size, n_layers, hidden_size, drop_prob):
        
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.drop_prob = drop_prob
                
        self.lstm = nn.LSTM(input_size, hidden_size[0], n_layers, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        # First fully connected layer
        self.fc1 = nn.Linear(hidden_size[0], hidden_size[1])
        # Second fully connected layer
        self.fc2 = nn.Linear(hidden_size[1], hidden_size[2])
        
        # Softmax activation
        self.softmax = nn.Softmax()
        
    def forward(self, x, hidden):
        # x(batch_size, seq_length, input_size)
        # hidden(n_layers, batch_size, hidden_dim)
        # r_out(batch_size, time_step, hidden_size)
        
        # Use x.view(seq_len, batch, input_size)
        # And this order corresponde to:
        # seq_len = time dimension (how many time steps do you have)
        # batch = mini-batch dimension (how much do you split ur data)
        # input_size = feature dimension (so how many features do you have)
        
        x = x.view(1, x.size(0), self.input_size)
        
        # get LSTM outputs
        lstm_out, hidden = self.lstm(x, hidden)
        # out = self.fc0(x)
        #print(out.shape)
        # shape output to be (batch_size*seq_length, hidden_dim)
        lstm_out = lstm_out.view(-1, self.hidden_size[0])
        
        # dropout layer
        out = self.dropout(lstm_out)
        
        # get output of first fully connected layer
        out1 = self.fc1(out)
        #print(out1.shape)
        
        # get output of second fully connected layer
        out2 = self.fc2(out1)
        #print(out2)
        
        # softmax activation function
        output = self.softmax(out2)
        #print(output.shape)
        
        # TODO: might need to reshape the out2
                
        return output, hidden
    
    def init_hidden(self, batch_size):
        '''initialize hidden states'''
        # initialized initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.paramaters()).data
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_size).zero_(),
                 weight.new(self.n_layers, batch_size, self.hidden_size).zero_())
        
        return hidden

In [25]:
def one_hot_encode(arr, n_labels):

    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

def preprocess(df):

    # drop ecgNum row
    df = df.drop(["ecgNum"], axis=1)
    # Classify the dependent and independent variables
    X = df.iloc[:, :-1].values
    Y = df.iloc[:, -1].values
    
    # split the data into train, validate, test
    X_temp, X_test, Y_temp, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    X_train, X_val, Y_train, Y_val = train_test_split(X_temp, Y_temp, test_size=0.2, random_state=0)
    
    return torch.from_numpy(X_train).float(), torch.from_numpy(X_val).float(), torch.from_numpy(X_test).float(), torch.from_numpy(Y_train), torch.from_numpy(Y_val), torch.from_numpy(Y_test)

In [26]:
#preprocess(data)

In [27]:
# size of the input at each time step
# TODO: handle input sizes
input_size = 100
# size of the hidden state and cell state at each time step
hidden_size = [64, 32, 6]
# TODO: ouptut sizes
output_size = [8, 4] # remove l8R
# number of LSTM layers
n_layers = 1
# dropout probability
drop_prob = 0.5

# instantiate the NN
neuralNet = NeuralNetwork(input_size, output_size, n_layers, hidden_size, drop_prob)
print(neuralNet)

NeuralNetwork(
  (lstm): LSTM(100, 64, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=6, bias=True)
  (softmax): Softmax(dim=None)
)


In [28]:
# Mean Squared Error and Adam Optimizer with a learning rate of 0.01
# TODO: play with LR
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(neuralNet.parameters(), lr=0.01)

In [34]:
# train the NN
def train(neuralNet, n_steps, print_every, clip):
    # initialize the hidden state
    # hidden = torch.from_numpy(np.zeros((100,1,1,64))).float()
    
    hidden = None
    
    X_train, X_val, X_test, Y_train, Y_val, Y_test = preprocess(data)
    
    for batch_i, step in enumerate(range(n_steps)):
                
        # zero accumulated gradients
        neuralNet.zero_grad()
        
        # output from the neural net
        prediction, hidden = neuralNet(X_train, hidden)
        
        print(prediction)
        print(Y_train)
        
        #hidden = hidden.data
        
         # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(neuralNet.parameters(), clip)
        optimizer.step()
        
        # calculate the loss
        loss = criterion(prediction, Y_train)
        # zero gradients
        optimizer.zero_grad()
        # backpropogate and update weights
        loss.backward()
        optimizer.step()
        
        # display loss and predictions
        if batch_i%print_every == 0:
            print('Loss ', loss.item())
            plt.plot(time_steps[1:], x, 'r.') #input
            plt.plot(time_steps[1:], predction.data.numpy().flatten(), 'b.') #predictions
            
    return neuralNet

In [35]:
n_steps = 100
print_every = 15
clip = 5

trained_NN = train(neuralNet, n_steps, print_every, clip)

c:\users\stidw\miniconda3\envs\deep-learning\lib\site-packages\ipykernel_launcher.py:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.1828, 0.1685, 0.1538, 0.1834, 0.1558, 0.1557],
        [0.1760, 0.1733, 0.1541, 0.1809, 0.1566, 0.1591],
        [0.1817, 0.1666, 0.1546, 0.1796, 0.1574, 0.1600],
        ...,
        [0.1825, 0.1703, 0.1543, 0.1804, 0.1542, 0.1583],
        [0.1848, 0.1693, 0.1527, 0.1819, 0.1550, 0.1563],
        [0.1802, 0.1701, 0.1551, 0.1854, 0.1511, 0.1583]],
       grad_fn=<SoftmaxBackward>)
tensor([0, 1, 0,  ..., 3, 0, 0])


c:\users\stidw\miniconda3\envs\deep-learning\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([64323])) that is different to the input size (torch.Size([64323, 6])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (6) must match the size of tensor b (64323) at non-singleton dimension 1